# Eşik Ayarlaması

👇 Çalışacağınız veri setini görmek için `player_performances.csv` oyuncu veri setini yükleyin.

In [15]:
import pandas as pd

#!curl -s https://d32aokrjazspmn.cloudfront.net/materials/ML_Player_performance.csv > data/player_performances.csv

data = pd.read_csv('ML_Player_performance.csv')

data.head()

,games played,minutes played,points per game,field goals made,field goal attempts,field goal percent,3 point made,3 point attempt,3 point %,free throw made,free throw attempts,free throw %,offensive rebounds,defensive rebounds,rebounds,assists,steals,blocks,turnovers,target_5y
0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0
2,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0
3,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1
4,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1


ℹ️ Her gözlem bir oyuncuyu temsil eder ve her sütun performansın bir özelliğidir. Hedef `target_5y`, oyuncunun 5 yıldan az [0] veya 5 yıl ve daha fazla [1] profesyonel kariyere sahip olup olmadığını tanımlar.

# Ön İşleme

👇 Ön işleme için çok fazla zaman harcamamak adına, tüm özellik setini Robust Scale ile ölçeklendirin. Bu uygulama optimal değildir, ancak ön işleme ve/veya modellerin hızla çalıştırılması için kullanılabilir.

Ölçeklendirilmiş özellik setini `X_scaled` olarak kaydedin.

In [16]:
from sklearn.preprocessing import RobustScaler

X = data.drop(columns=['target_5y'])
y = data['target_5y']

scaler = RobustScaler().set_output(transform='pandas')
X_scaled = scaler.fit_transform(X)

X_scaled.head()

,games played,minutes played,points per game,field goals made,field goal attempts,field goal percent,3 point made,3 point attempt,3 point %,free throw made,free throw attempts,free throw %,offensive rebounds,defensive rebounds,rebounds,assists,steals,blocks,turnovers
0,-0.900000,0.933884,0.352941,0.25,0.666667,-1.206557,1.00,1.500000,0.083077,0.585366,0.571429,-0.109375,-0.1,1.0625,0.659794,0.571429,-0.2,0.50,0.375
1,-0.933333,0.892562,0.313725,-0.05,0.452381,-1.875410,1.50,2.083333,0.036923,1.560976,1.357143,0.406250,-0.3,0.1875,-0.041237,1.857143,1.2,0.75,0.750
2,0.366667,-0.066116,-0.078431,-0.05,-0.023810,-0.222951,0.75,1.166667,0.064615,-0.097561,-0.142857,-0.335937,-0.3,0.0000,-0.123711,-0.071429,0.0,0.25,0.000
3,-0.166667,-0.371901,0.019608,0.10,0.166667,-0.170492,0.00,0.166667,0.009231,-0.097561,-0.142857,-0.187500,0.2,-0.5000,-0.247423,-0.214286,0.2,-0.25,0.000
4,-0.500000,-0.380165,-0.215686,-0.25,-0.428571,1.114754,-0.25,-0.166667,-0.686154,0.292683,0.285714,-0.304687,0.2,-0.1250,0.000000,-0.571429,-0.4,0.50,-0.250


### ☑️ Kodunuzu kontrol edin

In [17]:
from nbresult import ChallengeResult

result = ChallengeResult('scaled_features',
                         scaled_features = X_scaled
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/berkayturhan/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/berkayturhan/S16D3-S-data-threshold/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_scaled_features.py::TestScaled_features::test_scaled_features PASSED [100%]

============================== 1 passed in 0.60s ===============================


💯 You can commit your code:

git add tests/scaled_features.pickle

git commit -m 'Completed scaled_features step'

git push origin master



# Temel Modelleme

🎯 Görev, %90 garantiyle minimum 5 yıl profesyonel olarak devam edecek oyuncuları tespit etmektir.

👇 Varsayılan bir Lojistik Regresyon modeli antrenörün gereksinimlerini karşılayacak mı? Çapraz doğrulama kullanın ve cevabınızı destekleyen skoru `base_score` değişken adı altında kaydedin.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

model = LogisticRegression(max_iter=1000)
cv_results = cross_validate(model, X_scaled, y, cv=5, scoring='precision')

base_score = cv_results['test_score'].mean()
print(f'Standard Model Score: {base_score:.4f}')

Standard Model Score: 0.7379


### ☑️ Kodunuzu kontrol edin

In [19]:
from nbresult import ChallengeResult

result = ChallengeResult('base_precision',
                         score = base_score
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/berkayturhan/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/berkayturhan/S16D3-S-data-threshold/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_base_precision.py::TestBase_precision::test_precision_score PASSED  [100%]

============================== 1 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/base_precision.pickle

git commit -m 'Completed base_precision step'

git push origin master



# Eşik Ayarlaması

👇 Bir oyuncunun profesyonel olarak 5 yıl veya daha fazla sürmesi için %90 kesinlik garantisi veren karar eşiğini bulun. Eşiği `new_threshold` değişken adı altında kaydedin.

<details>
<summary>💡 İpucu</summary>

- [`cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) ile çapraz doğrulanmış olasılık tahminleri yapın
    
- Farklı eşiklerde kesinlik skorları oluşturmak için olasılıkları [`precision_recall_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html) içine yerleştirin

- 0.9 kesinliği garanti eden eşiği bulun
      
</details>

In [20]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
import numpy as np

y_scores = cross_val_predict(model, X_scaled, y, cv=5, method='predict_proba')[:, 1]

precisions, recalls, thresholds = precision_recall_curve(y, y_scores)

target_precision = 0.90
idx = np.argmax(precisions >= target_precision)

new_threshold = thresholds[idx]

print(f"Hedef Precision: {precisions[idx]:.4f}")
print(f"Gerekli Yeni Eşik Değeri: {new_threshold:.4f}")

Hedef Precision: 0.9010
Gerekli Yeni Eşik Değeri: 0.8667


### ☑️ Kodunuzu kontrol edin

In [21]:
from nbresult import ChallengeResult

result = ChallengeResult('decision_threshold',
                         threshold = new_threshold
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/berkayturhan/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/berkayturhan/S16D3-S-data-threshold/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_decision_threshold.py::TestDecision_threshold::test_new_threshold PASSED [100%]

============================== 1 passed in 0.12s ===============================


💯 You can commit your code:

git add tests/decision_threshold.pickle

git commit -m 'Completed decision_threshold step'

git push origin master



# Yeni Eşiği Kullanma

🎯 Antrenör potansiyel olarak ilginç bir oyuncu fark etti, ancak bu oyuncunun minimum 5 yıl profesyonel olarak devam edeceğine dair %90 garantinizi istiyor. Oyuncunun verilerini [buradan](https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_New_player.csv) indirin.

In [22]:
new_player = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/ML_New_player.csv")

new_player

,games played,minutes played,points per game,field goals made,field goal attempts,field goal percent,3 point made,3 point attempt,3 point %,free throw made,free throw attempts,free throw %,offensive rebounds,defensive rebounds,rebounds,assists,steals,blocks,turnovers
0,80,31.4,14.3,5.9,11.1,52.5,0.0,0.1,11.1,2.6,3.9,65.4,3.0,5.0,8.0,2.4,1.1,0.8,2.2


❓ Oyuncuyu antrenöre tavsiye etmeyi göze alır mısınız? Cevabınızı string olarak `recommendation` değişken adı altında "recommend" veya "not recommend" şeklinde kaydedin.

In [24]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)

new_player_scaled = scaler.transform(new_player)
log_model.fit(X_scaled, y)

probability = log_model.predict_proba(new_player_scaled)[0][1]

print(f"Başarı Olasılığı: {probability:.4f}")
print(f"Threshold: {new_threshold:.4f}")

if probability > new_threshold:
    recommendation = "recommend"
    print("Recommendation: Recommend the player.")
else:
    recommendation = "do not recommend"
    print("Recommendation: Do not recommend the player.")


Başarı Olasılığı: 0.9456
Threshold: 0.8667
Recommendation: Recommend the player.


### ☑️ Kodunuzu kontrol edin

In [25]:
from nbresult import ChallengeResult

result = ChallengeResult('recommendation',
                         recommendation = recommendation
)

result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/berkayturhan/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/berkayturhan/S16D3-S-data-threshold/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_recommendation.py::TestRecommendation::test_recommendation PASSED   [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/recommendation.pickle

git commit -m 'Completed recommendation step'

git push origin master



# 🏁